# Grade School Multiplication

In [1]:
import sys
from pathlib import Path

WORKSPACE = Path("..")

sys.path.insert(0, str(WORKSPACE / "src"))

In [2]:
from seqlogic import Vec, bits, cat, rep, u2bv

In [3]:
def add(a: Vec, b: Vec) -> Vec:
    """Ripple Carry Addition."""
    n = len(a)
    assert n > 0 and n == len(b)

    # Carries
    c = ["1b0"]
    for i, (a_i, b_i) in enumerate(zip(a, b)):
        c.append(a_i & b_i | c[i] & (a_i | b_i))
    c = cat(*c[:n])

    # Sum
    return a ^ b ^ c

In [4]:
def multiply(a: Vec, b: Vec) -> Vec:
    """Grade school multiplication."""
    n = len(a)
    assert n > 0 and n == len(b)

    # Calculate partial products
    pps = [rep(x, n) & a for x in b]

    # Accumulate partial product
    s = u2bv(0, 2 * n)
    for i, pp in enumerate(pps):
        s = add(s, cat(rep("1b0", i), pp, rep("1b0", n - i)))

    return s

In [ ]:
# Example: 7 * 6 = 42

#            0 1 1 1
#          X 0 1 1 0
#          ---------
#            0 0 0 0
#          0 1 1 1
#        0 1 1 1
#      0 0 0 0
#    ---------
#    0 0 1 0 1 0 1 0

multiply(bits("4b0111"), bits("4b0110"))

In [ ]:
size = 4
n = 1 << size

print("   | " + " ".join(f"{i:02x}" for i in range(n)))
print("---+" + ("---" * n))

for i in range(n):
    a = u2bv(i, size)
    print(f"{i:02x} |", end=" ")
    for j in range(n):
        b = u2bv(j, size)
        y = multiply(a, b).to_uint()

        # Verify the answer
        assert y == (i * j)

        print(f"{y:02x}", end="\n" if j == (n - 1) else " ")

In [ ]:
# Optimistic X propagation
multiply(bits("4b0000"), bits("4b----"))

In [ ]:
# Pessimistic X propagation
multiply(bits("4b0000"), bits("4bXXXX"))